In [1]:
!nvidia-smi

Sun May  5 00:23:19 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      On  | 00000000:00:03.0 Off |                    0 |
| N/A   70C    P8              20W /  72W |      4MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import os
HOME = os.getcwd()
print(HOME)

/home/jupyter/unattended-baggage-yolov8-modification


In [3]:
!pip install opencv-python
!pip install einops

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 8.5 MB/s eta 0:00:00


In [1]:
!pip install -q \
clearml

In [13]:
!pip install -e .

Obtaining file:///home/jupyter/unattended-baggage-yolov8-modification
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for ultralytics (pyproject.toml) ... done
  Created wheel for ultralytics: filename=ultralytics-8.2.2-0.editable-py3-none-any.whl size=16601 sha256=01a714cb67710d0bd4f7bed197a18bfae3defe2430d545d512f614a3413993b3
  Stored in directory: /var/tmp/pip-ephem-wheel-cache-0uxqm91l/wheels/16/24/4e/77eb3faafdc0df8f4050b5c10cf91dc4af4801edeb0d66c251
Successfully built ultralytics
  Attempting uninstall: ultralytics
    Found existing installation: ultralytics 8.2.2
    Uninstalling ultralytics-8.2.2:
      Successfully uninstalled ultralytics-8.2.2


## Setup ClearML

In [3]:
%env CLEARML_WEB_HOST=https://app.clear.ml
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml
%env CLEARML_API_ACCESS_KEY=3YPE676I4P8W2UIQKUCO
%env CLEARML_API_SECRET_KEY=dn1W9hkP09NFWXsATQENuBhUdol1ZrOVGFioO8ucf3kKKZJ2Q4

env: CLEARML_WEB_HOST=https://app.clear.ml
env: CLEARML_API_HOST=https://api.clear.ml
env: CLEARML_FILES_HOST=https://files.clear.ml
env: CLEARML_API_ACCESS_KEY=3YPE676I4P8W2UIQKUCO
env: CLEARML_API_SECRET_KEY=dn1W9hkP09NFWXsATQENuBhUdol1ZrOVGFioO8ucf3kKKZJ2Q4


## YOLOv8 baseline

In [5]:
from clearml import Task

# Step 1: Creating a ClearML Task
task = Task.init(
    project_name="Unattended Baggage Detection",
    task_name="1 May 2024 Model Training YOLOv8 baseline",
)

ClearML Task: created new task id=03cdd613e41a4de2ad14bfb18128491c
2024-05-01 02:56:36,982 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: https://app.clear.ml/projects/b0f06949b2114ff49e9e63675ea0f29b/experiments/03cdd613e41a4de2ad14bfb18128491c/output/log
ClearML Monitor: Could not detect iteration reporting, falling back to iterations as seconds-from-start


In [6]:
# Step 2: Selecting the YOLOv8 Model, for this case baseline, yolov8n
model_variant = "yolov8n"
task.set_parameter("model_variant", model_variant)

In [7]:
# # Step 3: Loading the YOLOv8 Model
from ultralytics import YOLO
model = YOLO(f'{model_variant}.pt')

2024-05-01 02:57:49,148 - clearml.model - INFO - Selected model id: 784b431658954ef096968ba8b9c4dedd


## YOLOv8 configuration used
```
model = yolov8n.pt
data = /home/jupyter/gcs/unattended-baggage/unattended_baggage_march_2024_experiment/data.yaml # Dataset config file
imgsz = 1280 # Image size
batch = 4  # Smaller batch size 
device = 0,1 # GPU device
epochs = 100 # Train for 100 epochs
workers = 2 # Lower workers for smaller GPU
optimizer = SGD # Default YOLO SGD
save_period = 5 # Save every 5 epochs
name = unattended_baggage_detection #experiment name
save = True #save train checkpoints and predict results
plots = True #save plots and images during train/val
patience = 25 #epochs to wait for no observable improvement for early stopping of training
verbose = True #whether to print verbose output
```
Rest use defaults for baseline training

In [8]:
# Step 4: Setting Up Training Arguments
args = dict(data='/home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/data.yaml',
                      imgsz=1280,
                      batch=4,
                      device = [0,1],
                      epochs=100,
                      workers = 2,
                      optimizer = 'SGD',
                      save_period = 5,
                      name = 'unattended_baggage_detection_yolov8_baseline',
                      save = True,
                      plots = True,
                      patience = 25,
                      verbose = True)
task.connect(args)

{'data': '/home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/data.yaml',
 'imgsz': 1280,
 'batch': 4,
 'device': [0, 1],
 'epochs': 100,
 'workers': 2,
 'optimizer': 'SGD',
 'save_period': 5,
 'name': 'unattended_baggage_detection_yolov8_baseline',
 'save': True,
 'plots': True,
 'patience': 25,
 'verbose': True}

In [9]:
# Step 5: Initiating Model Training
results = model.train(**args)

New https://pypi.org/project/ultralytics/8.2.5 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.2 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (NVIDIA L4, 22491MiB)
                                                            CUDA:1 (NVIDIA L4, 22491MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/data.yaml, epochs=100, time=None, patience=25, batch=4, imgsz=1280, save=True, save_period=5, cache=False, device=[0, 1], workers=2, project=None, name=unattended_baggage_detection_yolov8_baseline, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************


Ultralytics YOLOv8.2.2 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (NVIDIA L4, 22491MiB)
                                                            CUDA:1 (NVIDIA L4, 22491MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
Overriding model.yaml nc=80 with nc=3
Transferred 319/355 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅


train: Scanning /home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/train/labels.cache... 1159 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1159/1159 [00:00<?, ?it/s]
val: Scanning /home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/valid/labels.cache... 145 images, 0 backgrounds, 0 corrupt: 100%|██████████| 145/145 [00:00<?, ?it/s]


Plotting labels to /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_yolov8_baseline/labels.jpg... 
optimizer: SGD(lr=0.01, momentum=0.937) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 1280 train, 1280 val
Using 4 dataloader workers
Logging results to /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_yolov8_baseline
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        145        586      0.876      0.853      0.897      0.621

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    100/100      1.13G     0.7533     0.4267     0.9363          2       1280: 100%|██████████| 290/290 [00:45<00:00,  6.30it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 37/37 [00:02<00:00, 13.54it/s]


                   all        145        586      0.865      0.853      0.898      0.622

100 epochs completed in 1.386 hours.
Optimizer stripped from /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_yolov8_baseline/weights/last.pt, 6.3MB
Optimizer stripped from /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_yolov8_baseline/weights/best.pt, 6.3MB

Validating /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_yolov8_baseline/weights/best.pt...
Ultralytics YOLOv8.2.2 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (NVIDIA L4, 22491MiB)
                                                            CUDA:1 (NVIDIA L4, 22491MiB)
Model summary (fused): 168 layers, 3006233 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 37/37 [00:02<00:00, 12.81it/s]


                   all        145        586      0.878      0.857      0.906       0.63
              backpack        145        192      0.875      0.842      0.888      0.572
               handbag        145        310      0.844      0.872      0.911      0.526
              suitcase        145         84      0.916      0.857      0.919      0.793
Speed: 0.5ms preprocess, 13.8ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_yolov8_baseline
2024-05-01 04:29:13,604 - clearml.Task - INFO - Completed model upload to https://files.clear.ml/Unattended%20Baggage%20Detection/1%20May%202024%20Model%20Training%20YOLOv8%20baseline.03cdd613e41a4de2ad14bfb18128491c/models/best.pt


## YOLOv8 +AIFI backbone

In [4]:
from clearml import Task

# Step 1: Creating a ClearML Task
task = Task.init(
    project_name="Unattended Baggage Detection",
    task_name="3 May 2024 Model Training AIFI",
)


ClearML Task: created new task id=b297ea70df0f473bb52aba94536055e6
2024-05-03 00:27:53,778 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: https://app.clear.ml/projects/b0f06949b2114ff49e9e63675ea0f29b/experiments/b297ea70df0f473bb52aba94536055e6/output/log
ClearML Monitor: Could not detect iteration reporting, falling back to iterations as seconds-from-start


In [7]:
# Step 2: Selecting the YOLOv8 Model, for this case AIFI
model_variant = "ultralytics/cfg/models/v8/yolov8-aifi.yaml"
task.set_parameter("model_variant", model_variant)

In [8]:
# Step 3: Loading the YOLOv8 AIFI Model
from ultralytics import YOLO
# Load a model
model = YOLO(f'{model_variant}')
model.load('yolov8n.pt') # loading pretrain weights

WARNING ⚠️ no model scale passed. Assuming scale='n'.
2024-05-03 00:30:27,691 - clearml.model - INFO - Selected model id: 784b431658954ef096968ba8b9c4dedd
Transferred 343/355 items from pretrained weights


YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_s

## YOLOv8 AIFI Configuration Used
```
model = yolov8n.pt
data = /home/jupyter/gcs/unattended-baggage/unattended_baggage_march_2024_experiment/data.yaml # Dataset config file
imgsz = 1280 # Image size
batch = 4  # Smaller batch size for Tesla T4
device = 0,1 # GPU device
epochs = 100 # Train for 100 epochs
workers = 2 # Lower workers for smaller GPU
optimizer = SGD # Default YOLO SGD
save_period = 5 # Save every 5 epochs
name = unattended_baggage_detection_aifi #experiment name
save = True #save train checkpoints and predict results
plots = True #save plots and images during train/val
patience = 25 #epochs to wait for no observable improvement for early stopping of training
verbose = True #whether to print verbose output
```
Rest use defaults for training

In [10]:

# Step 4: Setting Up Training Arguments
args = dict(data='/home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/data.yaml',
                      imgsz=1280,
                      batch=4,
                      device = [0,1],
                      epochs=100,
                      workers = 2,
                      optimizer = 'SGD',
                      save_period = 5,
                      name = 'unattended_baggage_detection_aifi',
                      save = True,
                      plots = True,
                      patience = 25,
                      verbose = True)
task.connect(args)


{'data': '/home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/data.yaml',
 'imgsz': 1280,
 'batch': 4,
 'device': [0, 1],
 'epochs': 100,
 'workers': 2,
 'optimizer': 'SGD',
 'save_period': 5,
 'name': 'unattended_baggage_detection_aifi',
 'save': True,
 'plots': True,
 'patience': 25,
 'verbose': True}

In [12]:
# Step 5: Initiating Model Training
results = model.train(**args)

New https://pypi.org/project/ultralytics/8.2.6 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.2 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (NVIDIA L4, 22491MiB)
                                                            CUDA:1 (NVIDIA L4, 22491MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=ultralytics/cfg/models/v8/yolov8-aifi.yaml, data=/home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/data.yaml, epochs=100, time=None, patience=25, batch=4, imgsz=1280, save=True, save_period=5, cache=False, device=[0, 1], workers=2, project=None, name=unattended_baggage_detection_aifi3, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val,

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************


Ultralytics YOLOv8.2.2 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (NVIDIA L4, 22491MiB)
                                                            CUDA:1 (NVIDIA L4, 22491MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
Overriding model.yaml nc=80 with nc=3
WARNING ⚠️ no model scale passed. Assuming scale='n'.
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅


train: Scanning /home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/train/labels.cache... 1159 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1159/1159 [00:00<?, ?it/s]
val: Scanning /home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/valid/labels.cache... 145 images, 0 backgrounds, 0 corrupt: 100%|██████████| 145/145 [00:00<?, ?it/s]


Plotting labels to /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_aifi3/labels.jpg... 
optimizer: SGD(lr=0.01, momentum=0.937) with parameter groups 57 weight(decay=0.0), 66 weight(decay=0.0005), 67 bias(decay=0.0)
Image sizes 1280 train, 1280 val
Using 4 dataloader workers
Logging results to /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_aifi3
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        145        586      0.938      0.733      0.848      0.563

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    100/100      1.77G     0.9672     0.6709      1.179          2       1280: 100%|██████████| 290/290 [00:47<00:00,  6.12it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 37/37 [00:02<00:00, 13.74it/s]


                   all        145        586      0.921      0.738       0.85      0.564

100 epochs completed in 1.418 hours.
Optimizer stripped from /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_aifi3/weights/last.pt, 7.6MB
Optimizer stripped from /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_aifi3/weights/best.pt, 7.6MB

Validating /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_aifi3/weights/best.pt...
Ultralytics YOLOv8.2.2 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (NVIDIA L4, 22491MiB)
                                                            CUDA:1 (NVIDIA L4, 22491MiB)
YOLOv8-aifi summary (fused): 173 layers, 3631769 parameters, 0 gradients, 8.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 37/37 [00:02<00:00, 12.86it/s]


                   all        145        586      0.923      0.738      0.849      0.564
              backpack        145        192      0.905      0.766       0.85      0.535
               handbag        145        310      0.935      0.687      0.855      0.467
              suitcase        145         84      0.928      0.762      0.843       0.69
Speed: 0.5ms preprocess, 14.4ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_aifi3
2024-05-03 01:58:03,871 - clearml.Task - INFO - Completed model upload to https://files.clear.ml/Unattended%20Baggage%20Detection/3%20May%202024%20Model%20Training%20AIFI.b297ea70df0f473bb52aba94536055e6/models/best.pt


## YOLOv8 + BilevelRoutingAttention

In [4]:
from clearml import Task

# Step 1: Creating a ClearML Task
task = Task.init(
    project_name="Unattended Baggage Detection",
    task_name="3 May 2024 Model Training BiLevelRouting Attention",
)


ClearML Task: created new task id=616edbb8c0484e29a606b2d2c38ea22a
2024-05-03 08:11:28,572 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: https://app.clear.ml/projects/b0f06949b2114ff49e9e63675ea0f29b/experiments/616edbb8c0484e29a606b2d2c38ea22a/output/log
ClearML Monitor: Could not detect iteration reporting, falling back to iterations as seconds-from-start


In [5]:
# Step 2: Selecting the YOLOv8 Model, for this case BilevelRoutingAttention
model_variant = "ultralytics/cfg/models/v8/yolov8-BiLevelRoutingAttention.yaml"
task.set_parameter("model_variant", model_variant)

In [6]:
# Step 3: Loading the YOLOv8 BilevelRoutingAttention Model
from ultralytics import YOLO
# Load a model
model = YOLO(f'{model_variant}')
model.load('yolov8n.pt') # loading pretrain weights

WARNING ⚠️ no model scale passed. Assuming scale='n'.
2024-05-03 08:12:03,967 - clearml.model - INFO - Selected model id: 784b431658954ef096968ba8b9c4dedd
Transferred 186/379 items from pretrained weights


YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_s

## YOLOv8 BiLevelRouting Attention Configuration Used
```
model = yolov8n.pt
data = /home/jupyter/gcs/unattended-baggage/unattended_baggage_march_2024_experiment/data.yaml # Dataset config file
imgsz = 1280 # Image size
batch = 8  # Smaller batch size for Tesla T4
device = 0,1,2,3,4,5,6,7 # GPU device
epochs = 100 # Train for 100 epochs
workers = 2 # Lower workers for smaller GPU
optimizer = SGD # Default YOLO SGD
save_period = 5 # Save every 5 epochs
name = unattended_baggage_detection_bilevelroutingattention #experiment name
save = True #save train checkpoints and predict results
plots = True #save plots and images during train/val
patience = 25 #epochs to wait for no observable improvement for early stopping of training
verbose = True #whether to print verbose output
```
Rest use defaults for training

In [8]:
# Step 4: Setting Up Training Arguments
args = dict(data='/home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/data.yaml',
                      imgsz=1280,
                      batch=8,
                      device = [0,1,2,3,4,5,6,7],
                      epochs=100,
                      workers = 2,
                      optimizer = 'SGD',
                      save_period = 5,
                      name = 'unattended_baggage_detection_bilevelroutingattention_multihead',
                      save = True,
                      plots = True,
                      patience = 25,
                      verbose = True)
task.connect(args)

{'data': '/home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/data.yaml',
 'imgsz': 1280,
 'batch': 8,
 'device': [0, 1, 2, 3, 4, 5, 6, 7],
 'epochs': 100,
 'workers': 2,
 'optimizer': 'SGD',
 'save_period': 5,
 'name': 'unattended_baggage_detection_bilevelroutingattention_multihead',
 'save': True,
 'plots': True,
 'patience': 25,
 'verbose': True}

In [9]:
# Step 5: Initiating Model Training
results = model.train(**args)

New https://pypi.org/project/ultralytics/8.2.7 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.2 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (NVIDIA L4, 22491MiB)
                                                            CUDA:1 (NVIDIA L4, 22491MiB)
                                                            CUDA:2 (NVIDIA L4, 22491MiB)
                                                            CUDA:3 (NVIDIA L4, 22491MiB)
                                                            CUDA:4 (NVIDIA L4, 22491MiB)
                                                            CUDA:5 (NVIDIA L4, 22491MiB)
                                                            CUDA:6 (NVIDIA L4, 22491MiB)
                                                            CUDA:7 (NVIDIA L4, 22491MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=ultralytics/cfg/models/v8/yolov8-BiLevelRoutingAttention.yaml, data=/home/jupyt

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************


Ultralytics YOLOv8.2.2 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (NVIDIA L4, 22491MiB)
                                                            CUDA:1 (NVIDIA L4, 22491MiB)
                                                            CUDA:2 (NVIDIA L4, 22491MiB)
                                                            CUDA:3 (NVIDIA L4, 22491MiB)
                                                            CUDA:4 (NVIDIA L4, 22491MiB)
                                                            CUDA:5 (NVIDIA L4, 22491MiB)
                                                            CUDA:6 (NVIDIA L4, 22491MiB)
                                                            CUDA:7 (NVIDIA L4, 22491MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
Overriding model.yaml nc=80 with nc=3
WARNING ⚠️ no model scale passed. Assuming scale='n'.
Freezing layer 'model.26.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅


train: Scanning /home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/train/labels.cache... 1159 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1159/1159 [00:00<?, ?it/s]
val: Scanning /home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/valid/labels.cache... 145 images, 0 backgrounds, 0 corrupt: 100%|██████████| 145/145 [00:00<?, ?it/s]


Plotting labels to /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_bilevelroutingattention_multihead/labels.jpg... 
optimizer: SGD(lr=0.01, momentum=0.937) with parameter groups 57 weight(decay=0.0), 76 weight(decay=0.0005), 75 bias(decay=0.0)
Image sizes 1280 train, 1280 val
Using 16 dataloader workers
Logging results to /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_bilevelroutingattention_multihead
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        145        586      0.789      0.668      0.708      0.446
Closing dataloader mosaic

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     91/100      8.49G       1.05     0.8027       1.23          1       1280: 100%|██████████| 145/145 [00:36<00:00,  3.96it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 73/73 [00:04<00:00, 15.41it/s]


                   all        145        586      0.744      0.662      0.681      0.406
EarlyStopping: Training stopped early as no improvement observed in last 25 epochs. Best results observed at epoch 66, best model saved as best.pt.
To update EarlyStopping(patience=25) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

91 epochs completed in 1.042 hours.
Optimizer stripped from /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_bilevelroutingattention_multihead/weights/last.pt, 7.2MB
Optimizer stripped from /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_bilevelroutingattention_multihead/weights/best.pt, 7.2MB

Validating /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_bilevelroutingattention_multihead/weights/best.pt...
Ultralytics YOLOv8.2.2 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (NVIDIA L4, 22491MiB)

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 73/73 [00:04<00:00, 16.85it/s]


                   all        145        586      0.855      0.698      0.802      0.495
              backpack        145        192      0.802      0.719       0.79      0.457
               handbag        145        310      0.866      0.635       0.78      0.388
              suitcase        145         84      0.897      0.738      0.836      0.641
Speed: 0.4ms preprocess, 25.7ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_bilevelroutingattention_multihead
2024-05-03 09:17:18,404 - clearml.Task - INFO - Completed model upload to https://files.clear.ml/Unattended%20Baggage%20Detection/3%20May%202024%20Model%20Training%20BiLevelRouting%20Attention.616edbb8c0484e29a606b2d2c38ea22a/models/best.pt


### YOLOv8 + Shuffle Attention

In [11]:
from clearml import Task

# Step 1: Creating a ClearML Task
task = Task.init(
    project_name="Unattended Baggage Detection",
    task_name="Shuffle Attention 3 May 2024 Model Training Multi Head",
)


ClearML Task: created new task id=ef6340197a6e45ceaf83fe6ffab81685
ClearML results page: https://app.clear.ml/projects/b0f06949b2114ff49e9e63675ea0f29b/experiments/ef6340197a6e45ceaf83fe6ffab81685/output/log
ClearML Monitor: Could not detect iteration reporting, falling back to iterations as seconds-from-start


In [12]:
# Step 2: Selecting the YOLOv8 Model, for this case ShuffleAttention
model_variant = "ultralytics/cfg/models/v8/yolov8-ShuffleAttention.yaml"
task.set_parameter("model_variant", model_variant)

In [13]:
# Step 3: Loading the YOLOv8 ShuffleAttention Model
from ultralytics import YOLO
# Load a model
model = YOLO(f'{model_variant}')
model.load('yolov8n.pt') # loading pretrain weights

WARNING ⚠️ no model scale passed. Assuming scale='n'.


Connecting multiple input models with the same name: `yolov8n`. This might result in the wrong model being used when executing remotely


Transferred 186/379 items from pretrained weights


YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_s

## YOLOv8 Shuffle Attention Configuration Used
```
model = yolov8n.pt
data = /home/jupyter/gcs/unattended-baggage/unattended_baggage_march_2024_experiment/data.yaml # Dataset config file
imgsz = 1280 # Image size
batch = 4  # Smaller batch size for Tesla T4
device = 0,1 # GPU device
epochs = 100 # Train for 100 epochs
workers = 2 # Lower workers for smaller GPU
optimizer = SGD # Default YOLO SGD
save_period = 5 # Save every 5 epochs
name = unattended_baggage_detection_shuffleattention #experiment name
save = True #save train checkpoints and predict results
plots = True #save plots and images during train/val
patience = 25 #epochs to wait for no observable improvement for early stopping of training
verbose = True #whether to print verbose output
```
Rest use defaults for training

In [14]:
# Step 4: Setting Up Training Arguments
args = dict(data='/home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/data.yaml',
                      imgsz=1280,
                      batch=8,
                      device = [0,1,2,3,4,5,6,7],
                      epochs=100,
                      workers = 2,
                      optimizer = 'SGD',
                      save_period = 5,
                      name = 'unattended_baggage_detection_shuffleattention_multihead',
                      save = True,
                      plots = True,
                      patience = 25,
                      verbose = True)
task.connect(args)

{'data': '/home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/data.yaml',
 'imgsz': 1280,
 'batch': 8,
 'device': [0, 1, 2, 3, 4, 5, 6, 7],
 'epochs': 100,
 'workers': 2,
 'optimizer': 'SGD',
 'save_period': 5,
 'name': 'unattended_baggage_detection_shuffleattention_multihead',
 'save': True,
 'plots': True,
 'patience': 25,
 'verbose': True}

In [15]:
# Step 5: Initiating Model Training for ShuffleAttention
results = model.train(**args)

New https://pypi.org/project/ultralytics/8.2.7 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.2 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (NVIDIA L4, 22491MiB)
                                                            CUDA:1 (NVIDIA L4, 22491MiB)
                                                            CUDA:2 (NVIDIA L4, 22491MiB)
                                                            CUDA:3 (NVIDIA L4, 22491MiB)
                                                            CUDA:4 (NVIDIA L4, 22491MiB)
                                                            CUDA:5 (NVIDIA L4, 22491MiB)
                                                            CUDA:6 (NVIDIA L4, 22491MiB)
                                                            CUDA:7 (NVIDIA L4, 22491MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=ultralytics/cfg/models/v8/yolov8-ShuffleAttention.yaml, data=/home/jupyter/gcs/

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************


Ultralytics YOLOv8.2.2 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (NVIDIA L4, 22491MiB)
                                                            CUDA:1 (NVIDIA L4, 22491MiB)
                                                            CUDA:2 (NVIDIA L4, 22491MiB)
                                                            CUDA:3 (NVIDIA L4, 22491MiB)
                                                            CUDA:4 (NVIDIA L4, 22491MiB)
                                                            CUDA:5 (NVIDIA L4, 22491MiB)
                                                            CUDA:6 (NVIDIA L4, 22491MiB)
                                                            CUDA:7 (NVIDIA L4, 22491MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
WARNING ⚠️ no model scale passed. Assuming scale='n'.
Freezing layer 'model.26.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅


train: Scanning /home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/train/labels.cache... 1159 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1159/1159 [00:00<?, ?it/s]
val: Scanning /home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/valid/labels.cache... 145 images, 0 backgrounds, 0 corrupt: 100%|██████████| 145/145 [00:00<?, ?it/s]


Plotting labels to /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_shuffleattention_multihead/labels.jpg... 
optimizer: SGD(lr=0.01, momentum=0.937) with parameter groups 61 weight(decay=0.0), 72 weight(decay=0.0005), 75 bias(decay=0.0)
Image sizes 1280 train, 1280 val
Using 16 dataloader workers
Logging results to /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_shuffleattention_multihead
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

     94/100     0.799G     0.9945     0.7234      1.182          1       1280: 100%|██████████| 145/145 [00:32<00:00,  4.45it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 73/73 [00:04<00:00, 15.02it/s]


                   all        145        586       0.84      0.672      0.777      0.485
EarlyStopping: Training stopped early as no improvement observed in last 25 epochs. Best results observed at epoch 69, best model saved as best.pt.
To update EarlyStopping(patience=25) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

94 epochs completed in 0.968 hours.
Optimizer stripped from /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_shuffleattention_multihead/weights/last.pt, 6.3MB
Optimizer stripped from /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_shuffleattention_multihead/weights/best.pt, 6.3MB

Validating /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_shuffleattention_multihead/weights/best.pt...
Ultralytics YOLOv8.2.2 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (NVIDIA L4, 22491MiB)
                    

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 73/73 [00:04<00:00, 18.11it/s]


                   all        145        586      0.816      0.716      0.786      0.487
              backpack        145        192      0.856      0.713      0.769      0.437
               handbag        145        310      0.708        0.7      0.776      0.399
              suitcase        145         84      0.885      0.735      0.814      0.626
Speed: 0.4ms preprocess, 23.4ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_shuffleattention_multihead
2024-05-03 10:23:24,471 - clearml.Task - INFO - Completed model upload to https://files.clear.ml/Unattended%20Baggage%20Detection/Shuffle%20Attention%203%20May%202024%20Model%20Training%20Multi%20Head.ef6340197a6e45ceaf83fe6ffab81685/models/best.pt


Connecting multiple input models with the same name: `best`. This might result in the wrong model being used when executing remotely


## YOLOv8 + SPD-Conv

In [4]:
from clearml import Task

# Step 1: Creating a ClearML Task
task = Task.init(
    project_name="Unattended Baggage Detection",
    task_name="SPD-Conv 3 May 2024 Model Training",
)


ClearML Task: created new task id=b0a452ee45824f56808beefe8568c09c
2024-05-03 03:51:04,343 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: https://app.clear.ml/projects/b0f06949b2114ff49e9e63675ea0f29b/experiments/b0a452ee45824f56808beefe8568c09c/output/log
ClearML Monitor: Could not detect iteration reporting, falling back to iterations as seconds-from-start


In [6]:
# Step 2: Selecting the YOLOv8 Model, for this case SPD-Conv
model_variant = "ultralytics/cfg/models/v8/yolov8-SPD-Conv.yaml"
task.set_parameter("model_variant", model_variant)

In [8]:
# Step 3: Loading the YOLOv8 SPD-Conv Model
from ultralytics import YOLO
# Load a model
model = YOLO(f'{model_variant}')
model.load('yolov8n.pt') # loading pretrain weights

WARNING ⚠️ no model scale passed. Assuming scale='n'.
2024-05-03 03:51:49,285 - clearml.model - WARNING - Connecting multiple input models with the same name: `yolov8n`. This might result in the wrong model being used when executing remotely
Transferred 29/355 items from pretrained weights


YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): space_to_depth()
      (3): C2f(
        (cv1): Conv(
          (conv): Conv2d(128, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03,

In [9]:
# Step 4: Setting Up Training Arguments
args = dict(data='/home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/data.yaml',
                      imgsz=1280,
                      batch=4,
                      device = [0,1],
                      epochs=100,
                      workers = 2,
                      optimizer = 'SGD',
                      save_period = 5,
                      name = 'unattended_baggage_detection_SPD-Conv',
                      save = True,
                      plots = True,
                      patience = 25,
                      verbose = True)
task.connect(args)

{'data': '/home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/data.yaml',
 'imgsz': 1280,
 'batch': 4,
 'device': [0, 1],
 'epochs': 100,
 'workers': 2,
 'optimizer': 'SGD',
 'save_period': 5,
 'name': 'unattended_baggage_detection_SPD-Conv',
 'save': True,
 'plots': True,
 'patience': 25,
 'verbose': True}

In [10]:
# Step 5: Initiating Model Training for SPD-Conv
results = model.train(**args)

New https://pypi.org/project/ultralytics/8.2.7 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.2 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (NVIDIA L4, 22491MiB)
                                                            CUDA:1 (NVIDIA L4, 22491MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=ultralytics/cfg/models/v8/yolov8-SPD-Conv.yaml, data=/home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/data.yaml, epochs=100, time=None, patience=25, batch=4, imgsz=1280, save=True, save_period=5, cache=False, device=[0, 1], workers=2, project=None, name=unattended_baggage_detection_SPD-Conv, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, spl

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************


Ultralytics YOLOv8.2.2 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (NVIDIA L4, 22491MiB)
                                                            CUDA:1 (NVIDIA L4, 22491MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
Overriding model.yaml nc=80 with nc=3
WARNING ⚠️ no model scale passed. Assuming scale='n'.
Freezing layer 'model.28.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅


train: Scanning /home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/train/labels.cache... 1159 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1159/1159 [00:00<?, ?it/s]
val: Scanning /home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/valid/labels.cache... 145 images, 0 backgrounds, 0 corrupt: 100%|██████████| 145/145 [00:00<?, ?it/s]


Plotting labels to /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_SPD-Conv/labels.jpg... 
optimizer: SGD(lr=0.01, momentum=0.937) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 1280 train, 1280 val
Using 4 dataloader workers
Logging results to /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_SPD-Conv
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        145        586      0.921      0.737      0.853      0.569

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    100/100      5.45G     0.9274     0.6496       1.28          2       1280: 100%|██████████| 290/290 [01:59<00:00,  2.42it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 37/37 [00:05<00:00,  7.04it/s]


                   all        145        586      0.936      0.729      0.855      0.572

100 epochs completed in 3.516 hours.
Optimizer stripped from /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_SPD-Conv/weights/last.pt, 7.6MB
Optimizer stripped from /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_SPD-Conv/weights/best.pt, 7.6MB

Validating /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_SPD-Conv/weights/best.pt...
Ultralytics YOLOv8.2.2 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (NVIDIA L4, 22491MiB)
                                                            CUDA:1 (NVIDIA L4, 22491MiB)
YOLOv8-SPD-Conv summary (fused): 174 layers, 3451673 parameters, 0 gradients, 50.9 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 37/37 [00:04<00:00,  8.06it/s]


                   all        145        586       0.91      0.742      0.853      0.578
              backpack        145        192      0.908      0.781      0.837      0.529
               handbag        145        310      0.887      0.706      0.876      0.504
              suitcase        145         84      0.933      0.738      0.846      0.702
Speed: 0.5ms preprocess, 27.4ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_SPD-Conv
2024-05-03 07:24:37,966 - clearml.Task - INFO - Completed model upload to https://files.clear.ml/Unattended%20Baggage%20Detection/SPD-Conv%203%20May%202024%20Model%20Training.b0a452ee45824f56808beefe8568c09c/models/best.pt


## YOLOv8 + SPD-Conv + AIFI

In [4]:
from clearml import Task

# Step 1: Creating a ClearML Task
task = Task.init(
    project_name="Unattended Baggage Detection",
    task_name="SPD-Conv+AIFI 5 May 2024 Model Training",
)


ClearML Task: created new task id=80bebd3ebbd5460eb0b7081ceb7964d9
2024-05-05 00:23:51,292 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: https://app.clear.ml/projects/b0f06949b2114ff49e9e63675ea0f29b/experiments/80bebd3ebbd5460eb0b7081ceb7964d9/output/log
ClearML Monitor: Could not detect iteration reporting, falling back to iterations as seconds-from-start


In [5]:
# Step 2: Selecting the YOLOv8 Model, for this case AIFI + SPD-Conv
model_variant = "ultralytics/cfg/models/v8/yolov8-aifi-SPD-Conv.yaml"
task.set_parameter("model_variant", model_variant)

In [6]:
# Step 3: Loading the YOLOv8 AIFI + SPD-Conv Model
from ultralytics import YOLO
# Load a model
model = YOLO(f'{model_variant}')
model.load('yolov8n.pt') # loading pretrain weights

WARNING ⚠️ no model scale passed. Assuming scale='n'.
2024-05-05 00:24:05,806 - clearml.model - INFO - Selected model id: 784b431658954ef096968ba8b9c4dedd
Transferred 29/355 items from pretrained weights


YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): space_to_depth()
      (3): C2f(
        (cv1): Conv(
          (conv): Conv2d(128, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03,

In [7]:
# Step 4: Setting Up Training Arguments
args = dict(data='/home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/data.yaml',
                      imgsz=1280,
                      batch=8,
                      device = [0,1,2,3,4,5,6,7],
                      epochs=100,
                      workers = 2,
                      optimizer = 'SGD',
                      save_period = 5,
                      name = 'unattended_baggage_detection_AIFI_SPD-Conv',
                      save = True,
                      plots = True,
                      patience = 25,
                      verbose = True)
task.connect(args)

{'data': '/home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/data.yaml',
 'imgsz': 1280,
 'batch': 8,
 'device': [0, 1, 2, 3, 4, 5, 6, 7],
 'epochs': 100,
 'workers': 2,
 'optimizer': 'SGD',
 'save_period': 5,
 'name': 'unattended_baggage_detection_AIFI_SPD-Conv',
 'save': True,
 'plots': True,
 'patience': 25,
 'verbose': True}

In [8]:
# Step 5: Initiating Model Training for AIFI + SPD-Conv
results = model.train(**args)

New https://pypi.org/project/ultralytics/8.2.8 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.2 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (NVIDIA L4, 22491MiB)
                                                            CUDA:1 (NVIDIA L4, 22491MiB)
                                                            CUDA:2 (NVIDIA L4, 22491MiB)
                                                            CUDA:3 (NVIDIA L4, 22491MiB)
                                                            CUDA:4 (NVIDIA L4, 22491MiB)
                                                            CUDA:5 (NVIDIA L4, 22491MiB)
                                                            CUDA:6 (NVIDIA L4, 22491MiB)
                                                            CUDA:7 (NVIDIA L4, 22491MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=ultralytics/cfg/models/v8/yolov8-aifi-SPD-Conv.yaml, data=/home/jupyter/gcs/una

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************


Ultralytics YOLOv8.2.2 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (NVIDIA L4, 22491MiB)
                                                            CUDA:1 (NVIDIA L4, 22491MiB)
                                                            CUDA:2 (NVIDIA L4, 22491MiB)
                                                            CUDA:3 (NVIDIA L4, 22491MiB)
                                                            CUDA:4 (NVIDIA L4, 22491MiB)
                                                            CUDA:5 (NVIDIA L4, 22491MiB)
                                                            CUDA:6 (NVIDIA L4, 22491MiB)
                                                            CUDA:7 (NVIDIA L4, 22491MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
WARNING ⚠️ no model scale passed. Assuming scale='n'.
Freezing layer 'model.28.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅


train: Scanning /home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/train/labels.cache... 1159 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1159/1159 [00:00<?, ?it/s]
val: Scanning /home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/valid/labels.cache... 145 images, 0 backgrounds, 0 corrupt: 100%|██████████| 145/145 [00:00<?, ?it/s]


Plotting labels to /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_AIFI_SPD-Conv2/labels.jpg... 
optimizer: SGD(lr=0.01, momentum=0.937) with parameter groups 57 weight(decay=0.0), 66 weight(decay=0.0005), 67 bias(decay=0.0)
Image sizes 1280 train, 1280 val
Using 16 dataloader workers
Logging results to /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_AIFI_SPD-Conv2
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        145        586      0.881      0.688      0.799      0.535

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    100/100      7.65G     0.8479     0.6028      1.219          2       1280: 100%|██████████| 145/145 [00:54<00:00,  2.67it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 73/73 [00:05<00:00, 12.53it/s]


                   all        145        586      0.888      0.687      0.804      0.541

100 epochs completed in 1.687 hours.
Optimizer stripped from /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_AIFI_SPD-Conv2/weights/last.pt, 8.8MB
Optimizer stripped from /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_AIFI_SPD-Conv2/weights/best.pt, 8.8MB

Validating /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_AIFI_SPD-Conv2/weights/best.pt...
Ultralytics YOLOv8.2.2 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (NVIDIA L4, 22491MiB)
                                                            CUDA:1 (NVIDIA L4, 22491MiB)
                                                            CUDA:2 (NVIDIA L4, 22491MiB)
                                                            CUDA:3 (NVIDIA L4, 22491MiB)
                                                            CUDA:4

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 73/73 [00:05<00:00, 12.99it/s]


                   all        145        586        0.9       0.73       0.83      0.552
              backpack        145        192      0.837      0.724      0.808      0.498
               handbag        145        310      0.896      0.732      0.834      0.478
              suitcase        145         84      0.969      0.733      0.849       0.68
Speed: 0.4ms preprocess, 34.3ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_AIFI_SPD-Conv2
2024-05-05 02:07:02,223 - clearml.Task - INFO - Completed model upload to https://files.clear.ml/Unattended%20Baggage%20Detection/SPD-Conv%2BAIFI%205%20May%202024%20Model%20Training.80bebd3ebbd5460eb0b7081ceb7964d9/models/best.pt
